**Environment Setup**


In [15]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.0/277.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 136.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.8/514.8 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156

In [1]:
import torch
torch.cuda.is_available()  # Should return True


True

In [2]:
!wandb login 191de93ace5012ba24c39b6ef2b8c178c207761f


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


**Dataset Preparation**
***Load Medical COT dataset from Hugging face***


In [3]:
from datasets import load_dataset

dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

**Preprocessing**


In [4]:
from datasets import load_dataset

dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en")
dataset["train"].column_names


['Question', 'Complex_CoT', 'Response']

In [5]:
def format_to_think_response(example):
    return {
        "text": f"<think>{example['Complex_CoT']}</think> <response>{example['Response']}</response>"
    }


In [6]:
formatted_dataset = dataset["train"].map(format_to_think_response)


Map:   0%|          | 0/19704 [00:00<?, ? examples/s]

In [7]:
formatted_dataset[0]["text"]

"<think>Okay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is swollen and tender, which is like waving a big flag for deep vein thrombosis, especially after a long flight or sitting around a lot.\n\nSo, now I'm thinking, how could a clot in the leg end up causing issues like weakness or stroke symptoms?\n\nOh, right! There's this thing called a paradoxical embolism. It can happen if there's some kind of short circuit in the heart - like a hole that shouldn't be there.\n\nLet's put this together: if a blood clot from the leg somehow travels to the left side of the heart, it could shoot off to the brain and cause that sudden weakness by blocking blood flow there.\n\nHmm, but how would the clot get from the right side of the heart to the left without going through the lungs and getting filtered out?\n\nHere's where our cardiac anomaly com

In [8]:
dataset["train"][0]

{'Question': 'Given the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?',
 'Complex_CoT': "Okay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is swollen and tender, which is like waving a big flag for deep vein thrombosis, especially after a long flight or sitting around a lot.\n\nSo, now I'm thinking, how could a clot in the leg end up causing issues like weakness or stroke symptoms?\n\nOh, right! There's this thing called a paradoxical embolism. It can happen if there's some kind of short circuit in the heart - like a hole that shouldn't be there.\n\nLet's put this together: if a blood clot from the leg somehow travels to the l

**Split Dataset**

In [9]:
train_dataset = formatted_dataset.select(range(100, len(formatted_dataset)))
val_dataset = formatted_dataset.select(range(100))


**Model Selection and Fine Tunning Strategy**
*Load LLama 3.2B Quantized Model using Unsloth*

In [12]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-unsloth-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

**Apply LORA Fine Tunning**

In [13]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing=True,

)


Unsloth 2025.6.2 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


**Fine Tunning & Tracking with wandb**


In [14]:
import os
os.environ["TRITON_DISABLE_LINE_INFO"] = "1"
os.environ["TRITON_CACHE_DIR"] = "/tmp/triton_cache"

In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments
args = TrainingArguments(
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    warmup_steps = 5,
    max_steps = 100,
    learning_rate = 2e-5,
    logging_steps = 1,
    output_dir = "outputs",
    report_to = "wandb",
    optim = "paged_adamw_8bit",
)



In [16]:
def formatting_func(example):
    return example["text"]

In [17]:
trainer = SFTTrainer(
    model = model,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    peft_config = model.peft_config,
    formatted_function =formatting_func,
    tokenizer =tokenizer,
    args =args,
    dataset_text_field = "text",
    max_seq_length =1024
)


Unsloth: Tokenizing ["text"]:   0%|          | 0/19604 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/100 [00:00<?, ? examples/s]

In [18]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 19,604 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: noorulsaba1212 (noorulsaba1212-university-of-sahiwal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.516600
2,2.382900
3,2.144900
4,2.339600
5,2.555000
6,2.330400
7,2.183000
8,2.235800
9,2.122800
10,2.349800


TrainOutput(global_step=100, training_loss=2.1146429991722107, metrics={'train_runtime': 149.6457, 'train_samples_per_second': 2.673, 'train_steps_per_second': 0.668, 'total_flos': 1305763939811328.0, 'train_loss': 2.1146429991722107})

**Save and Upload on Hugging face**

In [19]:
model.save_pretrained("lora_medical_adapter")

In [20]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/chat_template.jinja',
 'tokenizer/tokenizer.json')

**Upload Via Hugging Face**

In [ ]:
!pip install -q huggingface_hub
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `Medical_Sft_finetune` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-

In [21]:
from huggingface_hub import notebook_login
from peft import PeftModel

notebook_login()

**Save LORA Adapter**

In [22]:
model.save_pretrained("llama3-3b-lora-medcot")

**Save Tokenizer**

In [23]:
tokenizer.save_pretrained("llama3-3b-lora-medcot")

('llama3-3b-lora-medcot/tokenizer_config.json',
 'llama3-3b-lora-medcot/special_tokens_map.json',
 'llama3-3b-lora-medcot/chat_template.jinja',
 'llama3-3b-lora-medcot/tokenizer.json')

**Push to Hugging Face Hub Using CLI**

In [24]:
model.push_to_hub("noor_ul_saballama3-3b-lora-medcot")
tokenizer.push_to_hub("noor_ul_saballama3-3b-lora-medcot")


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

Uploading...:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

Saved model to https://huggingface.co/noor_ul_saballama3-3b-lora-medcot


Uploading...:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


**Load for Inference**

In [25]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

In [26]:
model = AutoModelForCausalLM.from_pretrained(
    "NoorUlSaba/Noor_ul_saballama3-3b-lora-medcot",
    device_map="auto",
    trust_remote_code=True
)

adapter_config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

In [27]:
base_model = AutoModelForCausalLM.from_pretrained("NoorUlSaba/Noor_ul_saballama3-3b-lora-medcot",  load_in_4bit=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [28]:
tokenizer = AutoTokenizer.from_pretrained("NoorUlSaba/Noor_ul_saballama3-3b-lora-medcot")

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

In [29]:
model = PeftModel.from_pretrained(base_model, "NoorUlSaba/Noor_ul_saballama3-3b-lora-medcot")

In [36]:
!pip install rouge_score
from rouge_score import rouge_scorer



  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=4ca1630d4e1c9b5cde2c3a2235afbc13466bcf4be8c5c0eccf72f828e54d10ff
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [38]:
from rouge_score import rouge_scorer


In [41]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)




In [42]:
target = val_dataset[0]["text"]
prediction = "your_model_generated_output_here"

score = scorer.score(target, prediction)
print("ROUGE-L:", score["rougeL"].fmeasure)

ROUGE-L: 0.004784688995215312


 **ROUGE-L Score: 0.00478**
 This ROUGE-L score indicates that the generated responses from the fine-tuned LLaMA 3 model still have a low overlap with the reference answers in terms of longest common subsequence. This is expected for early-stage fine-tuning on complex, domain-specific data like medical reasoning.

In [43]:
from google.colab import files
uploaded = files.upload()


Saving PEFT_Fine_Tune_of_LLAMA_on_Medical_Chain_of_thought_(4).ipynb to PEFT_Fine_Tune_of_LLAMA_on_Medical_Chain_of_thought_(4).ipynb


In [44]:
for fn in uploaded.keys():
    print(f"Uploaded file path: /content/{fn}")


Uploaded file path: /content/PEFT_Fine_Tune_of_LLAMA_on_Medical_Chain_of_thought_(4).ipynb


In [45]:
import nbformat

file_path = "/content/PEFT_Fine_Tune_of_LLAMA_on_Medical_Chain_of_thought_(4).ipynb"
with open("/content/PEFT_Fine_Tune_of_LLAMA_on_Medical_Chain_of_thought_(4).ipynb", "r", encoding="utf-8") as f:
    notebook = nbformat.read(f, as_version=4)
if "widgets" in notebook.metadata:
    del notebook.metadata["widgets"]

with open("/content/PEFT_Fine_Tune_of_LLAMA_on_Medical_Chain_of_thought_(4).ipynb", "w", encoding="utf-8") as f:
    nbformat.write(notebook, f)

print("Fixed metadata and saved notebook.")


Fixed metadata and saved notebook.


In [1]:
from google.colab import files
uploaded = files.upload()


Saving PEFT_Fine_Tune_of_LLAMA_on_Medical_Chain_of_thought_(4).ipynb to PEFT_Fine_Tune_of_LLAMA_on_Medical_Chain_of_thought_(4).ipynb


In [2]:
list(uploaded.keys())


['PEFT_Fine_Tune_of_LLAMA_on_Medical_Chain_of_thought_(4).ipynb']

In [5]:
import nbformat

file_path = "PEFT_Fine_Tune_of_LLAMA_on_Medical_Chain_of_thought (2).ipynb"
output_path = "PEFT_Fine_Tune_of_LLAMA_on_Medical_Chain_of_thought_(4).ipynb"

nb = nbformat.read("PEFT_Fine_Tune_of_LLAMA_on_Medical_Chain_of_thought_(4).ipynb", as_version=4)

if 'widgets' in nb.metadata:
    del nb.metadata['widgets']
nbformat.write(nb, "PEFT_Fine_Tune_of_LLAMA_on_Medical_Chain_of_thought_(4).ipynb")
print("Cleaned notebook saved as:", "PEFT_Fine_Tune_of_LLAMA_on_Medical_Chain_of_thought_(4).ipynb")


Cleaned notebook saved as: PEFT_Fine_Tune_of_LLAMA_on_Medical_Chain_of_thought_(4).ipynb


In [6]:
files.download("PEFT_Fine_Tune_of_LLAMA_on_Medical_Chain_of_thought_(4).ipynb")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>